<h1>Scraping Fidelity.com</h1>
In this assignment, you will scrape data from fidelity.com. The goal of the exercise is to get the latest sector performance data from the US markets, and to get the total market capitalization for each sector. 

The end result is to write a function: <i>get_us_sector_performance()</i> that will return a list of tuples. Each tuple should correspond to a sector and should contain the following data:
<li>the sector name
<li>the amount the sector has moved
<li>the market capitalization of the sector
<li>the market weight of the sector
<li>a link to the fidelity page for that sector

<p>
The data should be sorted by decreasing order of market weight. I.e., the sector with the highest weight should be in the first tuple, etc.

<h3>Process</h3>
<li>Get a list of sectors and the links to the sector detail pages from the url (see function)
<li>Loop through the list and call the function <i>get_sector_change_and_market_cap(sector_page_link)</i> for each sector
<li>Accumulate the name, the change, the capitalization, the weight and the link for each sector in output_list (see function)
<li>Sort the list by market weight

<b>Notes:</b>
<li>Note that the market weight is a string with a % sign at the back. You will need to get rid of the % and convert the string into a float before you can sort it
<li>Your starting data is the url listed below. You need to extract all data, including links to the sector pages, from the page at this url
<li>To sort a list of tuples by an arbitrary element, use the example at the bottom of this notebook

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
def get_us_sector_performance():
    output_list = list()
    url = "https://eresearch.fidelity.com/eresearch/goto/markets_sectors/landing.jhtml"
    short_url = "https://eresearch.fidelity.com"
        #try to access the web page
    try:
        response = requests.get(url)
        if not response.status_code == 200: #ALWAYS DO IT!!!!
            print("HTTP error",response.status_code)
    except:
        print("Something went wrong with requests.get")

        #Get a list of sectors and the links to the sector detail pages from the url 
    sector_list = []
    results_page = BeautifulSoup(response.content,'lxml')
    div_tag = results_page.find_all('div',{"class":"heading"})
    for div in div_tag:
        name = div.find('a',{"class":"heading1"})
        if name:
            link = name.get('href')
            link = short_url + str(link)
            name = name.get_text()

            #call get_sector_change_and_market_cap(sector_page_link)
            sector_change,sector_market_cap,sector_market_weight = get_sector_change_and_market_cap(link)

            #create the tuple and append to the list
            output_list.append((name, sector_change, sector_market_cap, sector_market_weight, link))
    output_list = sorted(output_list,key=lambda x: float(x[3]), reverse=True) 
    return output_list
    

In [3]:
def get_sector_change_and_market_cap(sector_page_link):
    
    #this is the function to call into the for loop of get_us_sector_performance()
    my_list=[]
    try:
        response = requests.get(sector_page_link)
        if not response.status_code == 200:
            print("HTTP error",response.status_code)
    except:
        print("Something went wrong with requests.get")
        return None
    results_page = BeautifulSoup(response.content,'lxml')
    table = results_page.find('table',{"class":"snapshot-data-tbl"}).find_all('td')
    for td in table:
        my_list.append(td.find('span').get_text())
    my_list = my_list[:-1]
    return my_list[0][:-1], my_list[1], float(my_list[2][:-1])

In [4]:
#Test get_sector_change_and_market_cap()
link = "https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=25"
get_sector_change_and_market_cap(link)
#Should return (-0.40, '$3.58T', 6.80)
#Note that neither the -0.40, nor the 6.80, end with a % sign

('-1.22', '$7.07T', 10.64)

In [5]:
#Test get_us_sector_performance()
get_us_sector_performance()

[('Information Technology',
  '-0.55',
  '$13.02T',
  27.23,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=45'),
 ('Health Care',
  '+0.94',
  '$7.68T',
  14.38,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=35'),
 ('Financials',
  '+0.40',
  '$8.33T',
  11.56,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=40'),
 ('Consumer Discretionary',
  '-1.22',
  '$7.07T',
  10.64,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=25'),
 ('Industrials',
  '+0.74',
  '$5.47T',
  8.34,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=20'),
 ('Communication Services',
  '-0.89',
  '$4.32T',
  8.16,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=lear

In [6]:
#Running the above cell should return (example: obviously the results will vary over time!)
"""
[('Information Technology',
  2.16,
  '$14.64T',
  28.02,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=45'),
 ('Health Care',
  0.19,
  '$7.60T',
  13.22,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=35'),
 ('Consumer Discretionary',
  1.96,
  '$8.32T',
  11.94,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=25'),
 ('Financials',
  0.63,
  '$8.83T',
  11.32,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=40'),
 ('Communication Services',
  0.78,
  '$5.74T',
  10.01,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=50'),
 ('Industrials',
  0.43,
  '$5.58T',
  8.06,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=20'),
 ('Consumer Staples',
  -0.16,
  '$4.26T',
  6.27,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=30'),
 ('Energy',
  0.12,
  '$3.19T',
  3.27,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=10'),
 ('Real Estate',
  0.69,
  '$1.71T',
  2.72,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=60'),
 ('Utilities',
  0.55,
  '$1.63T',
  2.6,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=55'),
 ('Materials',
  0.25,
  '$2.53T',
  2.58,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=15')]
  
"""

"\n[('Information Technology',\n  2.16,\n  '$14.64T',\n  28.02,\n  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=45'),\n ('Health Care',\n  0.19,\n  '$7.60T',\n  13.22,\n  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=35'),\n ('Consumer Discretionary',\n  1.96,\n  '$8.32T',\n  11.94,\n  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=25'),\n ('Financials',\n  0.63,\n  '$8.83T',\n  11.32,\n  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=40'),\n ('Communication Services',\n  0.78,\n  '$5.74T',\n  10.01,\n  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=50'),\n ('Industrials',\n  0.43,\n  '$5.58T',\n  8.06,\n  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market

In [7]:
#Print the different sector names from the above output in alpabetical order
names_ordered = []
for t in get_us_sector_performance():
    names_ordered.append(t[0])
names_ordered = sorted(names_ordered)
for i in names_ordered:
    print(i)

Communication Services
Consumer Discretionary
Consumer Staples
Energy
Financials
Health Care
Industrials
Information Technology
Materials
Real Estate
Utilities


In [8]:
### Find the sector with lowest Market cap by creating a dict of all sectors and their market caps
##    example of the output. {'Materials': '$2.01T', ...}
my_dict = dict()
output_list = sorted(get_us_sector_performance(),key=lambda x: float(x[2][1:-1]))
for t in output_list:
    my_dict[t[0]] = t[2]
minimum = min(my_dict, key=my_dict.get)
print(my_dict)
print("---")
print(f"Sector with Lowest Market cap: {minimum} with value {my_dict[minimum]}")

{'Real Estate': '$1.37T', 'Utilities': '$1.55T', 'Materials': '$2.50T', 'Energy': '$3.76T', 'Consumer Staples': '$4.17T', 'Communication Services': '$4.32T', 'Industrials': '$5.47T', 'Consumer Discretionary': '$7.07T', 'Health Care': '$7.68T', 'Financials': '$8.33T', 'Information Technology': '$13.02T'}
---
Sector with Lowest Market cap: Real Estate with value $1.37T


<h3>Hint: How to sort tuples by an arbitrary element?</h3>

In [9]:
x = [('a',23.2,'b'),('c',17.4,'f'),('d',29.2,'z'),('e',1.74,'bb')]
#Sort by the first element of the tuple


In [10]:
x = [('a',23.2,'b'),('c',17.4,'f'),('d',29.2,'z'),('e',1.74,'bb')]
#Sort by the element at position 1
